Section 1- Data Cleaning

In [153]:
import pandas as pd
import seaborn as sns

In [154]:
#Importing datasets
od1 = pd.read_excel('od1.xlsx', engine = 'openpyxl')
od2 = pd.read_excel('od2.xlsx', engine = 'openpyxl')
os = pd.read_excel('os.xlsx', engine = 'openpyxl')

Problem 1

In [155]:
#Merging datasets and sorting by ID
merged =pd.merge(od1, od2, how ='left', on=['dioptre_1', 'dioptre_2', 'Pachymetry', 'astigmatism', 'Diagnosis'])
sorted = merged.sort_values(by = ["ID"])

In [156]:
#Creating Sub-columns
def add_top_column(df, top_col, inplace=False):
    df.columns = pd.MultiIndex.from_product([[top_col], df.columns])
    return df
ID = sorted["ID"]
ID = pd.DataFrame(ID)
ID = add_top_column(ID, "")
Age = sorted["Age"]
Age = pd.DataFrame(Age)
Age = add_top_column(Age, "Age")
Gender = sorted["Gender"]
Gender = pd.DataFrame(Gender)
Gender = add_top_column(Gender, "Gender")
Diagnosis = sorted["Diagnosis"]
Diagnosis = pd.DataFrame(Diagnosis)
Diagnosis = add_top_column(Diagnosis, "Diagnosis")
Refractive_Defect = sorted[["dioptre_1", "dioptre_2", "astigmatism"]]
Refractive_Defect = pd.DataFrame(Refractive_Defect)
Refractive_Defect = add_top_column(Refractive_Defect, "Refractive_Defect")
PhakicPseudophakic = sorted["Phakic/Pseudophakic"]
PhakicPseudophakic = pd.DataFrame(PhakicPseudophakic)
PhakicPseudophakic = add_top_column(PhakicPseudophakic, "Phakic/Pseudophakic")
IOP = sorted[["Pneumatic","Perkins"]]
IOP = pd.DataFrame(IOP)
IOP = add_top_column(IOP, "IOP")
Pachymetry = sorted["Pachymetry"]
Pachymetry = pd.DataFrame(Pachymetry)
Pachymetry = add_top_column(Pachymetry, "Pachymetry")
Axial_Length = sorted["Axial_Length"]
Axial_Length = pd.DataFrame(Axial_Length)
Axial_Length = add_top_column(Axial_Length, "Axial_Length")
VF_MD = sorted["VF_MD"]
VF_MD = pd.DataFrame(VF_MD)
VF_MD = add_top_column(VF_MD, "VF_MD")
df_formatted = pd.concat([ID, Age, Gender, Diagnosis, Refractive_Defect, PhakicPseudophakic, IOP, Pachymetry,Axial_Length, VF_MD ], axis=1)
df_formatted.to_excel("df_formatted.xlsx")

Problem 2

In [157]:
#changing format of OS to match
tuples = [('', 'ID'), ('Age', 'Age'), ('Gender', 'Gender'), ('Diagnosis', 'Diagnosis'),
          ('Refractive_Defect', 'dioptre_1'), ('Refractive_Defect', 'dioptre_2'),
          ('Refractive_Defect', 'astigmatism'), ('Phakic/Pseudophakic', 'Phakic/Pseudophakic'),
          ('IOP', 'Pneumatic'), ('IOP', 'Perkins'), ('Pachymetry', 'Pachymetry'),
          ('Axial_Length', 'Axial_Length'), ('VF_MD', 'VF_MD')]
multi_index = pd.MultiIndex.from_tuples(tuples)
os.columns = multi_index

In [170]:
od_clean = df_formatted.copy(deep=True)
os_clean = os.copy(deep=True)

In [159]:
for column in od_clean.columns:
  od_clean_col = od_clean[column]
  column_dtype = od_clean_col.dtypes
  column_nas = od_clean_col.isna().sum()
  column_unique = len(od_clean_col.unique())
  print(f"Selected column: {column}")
  print(f"Data type: {column_dtype}")
  print(f"Number of NaNs: {column_nas}")
  print(f"Number of uniques: {column_unique}")
  print()

Selected column: ('', 'ID')
Data type: object
Number of NaNs: 0
Number of uniques: 244

Selected column: ('Age', 'Age')
Data type: int64
Number of NaNs: 0
Number of uniques: 53

Selected column: ('Gender', 'Gender')
Data type: object
Number of NaNs: 0
Number of uniques: 2

Selected column: ('Diagnosis', 'Diagnosis')
Data type: object
Number of NaNs: 0
Number of uniques: 3

Selected column: ('Refractive_Defect', 'dioptre_1')
Data type: float64
Number of NaNs: 15
Number of uniques: 43

Selected column: ('Refractive_Defect', 'dioptre_2')
Data type: float64
Number of NaNs: 3
Number of uniques: 21

Selected column: ('Refractive_Defect', 'astigmatism')
Data type: float64
Number of NaNs: 4
Number of uniques: 98

Selected column: ('Phakic/Pseudophakic', 'Phakic/Pseudophakic')
Data type: object
Number of NaNs: 5
Number of uniques: 3

Selected column: ('IOP', 'Pneumatic')
Data type: float64
Number of NaNs: 47
Number of uniques: 23

Selected column: ('IOP', 'Perkins')
Data type: float64
Number of

In [160]:
for column in os_clean.columns:
  os_clean_col = os_clean[column]
  column_dtype = os_clean_col.dtypes
  column_nas = os_clean_col.isna().sum()
  column_unique = len(os_clean_col.unique())
  print(f"Selected column: {column}")
  print(f"Data type: {column_dtype}")
  print(f"Number of NaNs: {column_nas}")
  print(f"Number of uniques: {column_unique}")
  print()

Selected column: ('', 'ID')
Data type: object
Number of NaNs: 0
Number of uniques: 245

Selected column: ('Age', 'Age')
Data type: object
Number of NaNs: 0
Number of uniques: 54

Selected column: ('Gender', 'Gender')
Data type: object
Number of NaNs: 0
Number of uniques: 3

Selected column: ('Diagnosis', 'Diagnosis')
Data type: object
Number of NaNs: 0
Number of uniques: 11

Selected column: ('Refractive_Defect', 'dioptre_1')
Data type: object
Number of NaNs: 12
Number of uniques: 44

Selected column: ('Refractive_Defect', 'dioptre_2')
Data type: object
Number of NaNs: 5
Number of uniques: 23

Selected column: ('Refractive_Defect', 'astigmatism')
Data type: object
Number of NaNs: 5
Number of uniques: 79

Selected column: ('Phakic/Pseudophakic', 'Phakic/Pseudophakic')
Data type: object
Number of NaNs: 6
Number of uniques: 4

Selected column: ('IOP', 'Pneumatic')
Data type: object
Number of NaNs: 45
Number of uniques: 23

Selected column: ('IOP', 'Perkins')
Data type: object
Number of Na

In [171]:
#Filling Na's, some with rounded means, some with 0
mean_dio1 = od_clean['Refractive_Defect','dioptre_1'].mean()
round_dio1 = round(mean_dio1 / 0.25) * 0.25
od_clean['Refractive_Defect', 'dioptre_1'] = od_clean['Refractive_Defect', 'dioptre_1'].fillna(round_dio1)

mean_dio2 = od_clean['Refractive_Defect','dioptre_2'].mean()
round_dio2 = round(mean_dio2 / 0.25) * 0.25
od_clean['Refractive_Defect', 'dioptre_2'] = od_clean['Refractive_Defect', 'dioptre_2'].fillna(round_dio2)

mean_ast = od_clean['Refractive_Defect','astigmatism'].mean()
round_ast = round(mean_ast)
od_clean['Refractive_Defect', 'astigmatism'] = od_clean['Refractive_Defect', 'astigmatism'].fillna(round_ast)

mode_pp = str(od_clean[('Phakic/Pseudophakic', 'Phakic/Pseudophakic')].mode().iloc[0])
od_clean['Phakic/Pseudophakic', 'Phakic/Pseudophakic'] = od_clean['Phakic/Pseudophakic', 'Phakic/Pseudophakic'].fillna(mode_pp)

mean_pne = od_clean['IOP', 'Pneumatic'].mean()
round_pne = round(mean_pne)
od_clean['IOP', 'Pneumatic'] = od_clean['IOP', 'Pneumatic'].fillna(round_pne)

mean_per = od_clean['IOP', 'Perkins'].mean()
round_per = round(mean_per)
od_clean['IOP', 'Perkins'] = od_clean['IOP', 'Perkins'].fillna(round_per)

mean_pach = od_clean['Pachymetry', 'Pachymetry'].mean()
round_pach = round(mean_pach)
od_clean['Pachymetry', 'Pachymetry'] = od_clean['Pachymetry', 'Pachymetry'].fillna(round_pach)


od_clean['Axial_Length', 'Axial_Length'] = od_clean['Axial_Length', 'Axial_Length'].fillna(26)

mean_vf = od_clean['VF_MD', 'VF_MD'].mean()
round_vf = round(mean_vf/0.01)*0.01
od_clean['VF_MD', 'VF_MD'] = od_clean['VF_MD', 'VF_MD'].fillna(round_vf)

od_clean.to_excel("od_clean.xlsx")

In [ ]:
print(os_clean.columns)
#Filling Na's, some with rounded means, some with 0
os_clean_dio1 = os_clean['Refractive_Defect','dioptre_1']
os_clean_dio1 = pd.to_numeric(os_clean_dio1, errors='coerce')
mean_dio1 = os_clean_dio1.mean()
round_dio1 = round(mean_dio1 / 0.25) * 0.25
os_clean['Refractive_Defect', 'dioptre_1'] = os_clean['Refractive_Defect', 'dioptre_1'].fillna(round_dio1)

mean_dio2 = os_clean['Refractive_Defect','dioptre_2'].mean()
round_dio2 = round(mean_dio2 / 0.25) * 0.25
os_clean['Refractive_Defect', 'dioptre_2'] = os_clean['Refractive_Defect', 'dioptre_2'].fillna(round_dio2)

mean_ast = os_clean['Refractive_Defect','astigmatism'].mean()
round_ast = round(mean_ast)
os_clean['Refractive_Defect', 'astigmatism'] = os_clean['Refractive_Defect', 'astigmatism'].fillna(round_ast)

mode_pp = str(os_clean[('Phakic/Pseudophakic', 'Phakic/Pseudophakic')].mode().iloc[0])
os_clean['Phakic/Pseudophakic', 'Phakic/Pseudophakic'] = os_clean['Phakic/Pseudophakic', 'Phakic/Pseudophakic'].fillna(mode_pp)

mean_pne = os_clean['IOP', 'Pneumatic'].mean()
round_pne = round(mean_pne)
os_clean['IOP', 'Pneumatic'] = os_clean['IOP', 'Pneumatic'].fillna(round_pne)

mean_per = os_clean['IOP', 'Perkins'].mean()
round_per = round(mean_per)
os_clean['IOP', 'Perkins'] = os_clean['IOP', 'Perkins'].fillna(round_per)

mean_pach = os_clean['Pachymetry', 'Pachymetry'].mean()
round_pach = round(mean_pach)
os_clean['Pachymetry', 'Pachymetry'] = os_clean['Pachymetry', 'Pachymetry'].fillna(round_pach)


os_clean['Axial_Length', 'Axial_Length'] = os_clean['Axial_Length', 'Axial_Length'].fillna(26)

mean_vf = os_clean['VF_MD', 'VF_MD'].mean()
round_vf = round(mean_vf/0.01)*0.01
os_clean['VF_MD', 'VF_MD'] = os_clean['VF_MD', 'VF_MD'].fillna(round_vf)

os_clean.to_excel("os_clean.xlsx")

In [180]:
#Same for os_clean, extra few lines as different format
os_clean_dio1 = os_clean['Refractive_Defect','dioptre_1']
os_clean_dio1 = pd.to_numeric(os_clean_dio1, errors='coerce')
mean_dio1 = os_clean_dio1.mean()
round_dio1 = round(mean_dio1 / 0.25) * 0.25
os_clean['Refractive_Defect', 'dioptre_1'] = os_clean['Refractive_Defect', 'dioptre_1'].fillna(round_dio1)

os_clean_dio2 = os_clean['Refractive_Defect','dioptre_2']
os_clean_dio2 = pd.to_numeric(os_clean_dio2, errors='coerce')
mean_dio2 = os_clean_dio2.mean()
round_dio2 = round(mean_dio2 / 0.25) * 0.25
os_clean['Refractive_Defect', 'dioptre_2'] = os_clean['Refractive_Defect', 'dioptre_2'].fillna(round_dio2)

os_clean_ast = os_clean['Refractive_Defect','astigmatism']
os_clean_ast = pd.to_numeric(os_clean_dio1, errors='coerce')
mean_ast = os_clean_ast.mean()
round_ast = round(mean_ast)
os_clean['Refractive_Defect', 'astigmatism'] = os_clean['Refractive_Defect', 'astigmatism'].fillna(round_ast)

mode_pp = str(os_clean[('Phakic/Pseudophakic', 'Phakic/Pseudophakic')].mode().iloc[0])
os_clean['Phakic/Pseudophakic', 'Phakic/Pseudophakic'] = os_clean['Phakic/Pseudophakic', 'Phakic/Pseudophakic'].fillna(mode_pp)

os_clean_pne = os_clean['IOP', 'Pneumatic']
os_clean_pne = pd.to_numeric(os_clean_pne, errors='coerce')
mean_pne = os_clean_pne.mean()
round_pne = round(mean_pne)
os_clean['IOP', 'Pneumatic'] = os_clean['IOP', 'Pneumatic'].fillna(round_pne)

os_clean_per = os_clean['IOP', 'Perkins']
os_clean_per = pd.to_numeric(os_clean_per, errors='coerce')
mean_per = os_clean_per.mean()
round_per = round(mean_per)
os_clean['IOP', 'Perkins'] = os_clean['IOP', 'Perkins'].fillna(round_per)

os_clean_pach = os_clean['Pachymetry', 'Pachymetry']
os_clean_pach = pd.to_numeric(os_clean_pach, errors='coerce')
mean_pach = os_clean_pach.mean()
round_pach = round(mean_pach)
os_clean['Pachymetry', 'Pachymetry'] = os_clean['Pachymetry', 'Pachymetry'].fillna(round_pach)


os_clean['Axial_Length', 'Axial_Length'] = os_clean['Axial_Length', 'Axial_Length'].fillna(26)

os_clean_vf = os_clean['VF_MD', 'VF_MD']
os_clean_vf = pd.to_numeric(os_clean_vf, errors='coerce')
mean_vf = os_clean_vf.mean()
round_vf = round(mean_vf/0.01)*0.01
os_clean['VF_MD', 'VF_MD'] = os_clean['VF_MD', 'VF_MD'].fillna(round_vf)

print(os_clean.iloc[0, :])
os_clean = os_clean.drop(os_clean.index[0])
os_clean.to_excel("os_clean.xlsx")


                     ID                                      ID
Age                  Age                                    Age
Gender               Gender                              Gender
Diagnosis            Diagnosis                        Diagnosis
Refractive_Defect    dioptre_1                        dioptre_1
                     dioptre_2                        dioptre_2
                     astigmatism                    astigmatism
Phakic/Pseudophakic  Phakic/Pseudophakic    Phakic/Pseudophakic
IOP                  Pneumatic                        Pneumatic
                     Perkins                            Perkins
Pachymetry           Pachymetry                      Pachymetry
Axial_Length         Axial_Length                  Axial_Length
VF_MD                VF_MD                                VF_MD
Name: 0, dtype: object


In [182]:
#Confirming no duplicates
duplicates = od_clean.duplicated()
print(od_clean[duplicates == True])
duplicates = os_clean.duplicated()
print(os_clean[duplicates == True])

Empty DataFrame
Columns: [(, ID), (Age, Age), (Gender, Gender), (Diagnosis, Diagnosis), (Refractive_Defect, dioptre_1), (Refractive_Defect, dioptre_2), (Refractive_Defect, astigmatism), (Phakic/Pseudophakic, Phakic/Pseudophakic), (IOP, Pneumatic), (IOP, Perkins), (Pachymetry, Pachymetry), (Axial_Length, Axial_Length), (VF_MD, VF_MD)]
Index: []
Empty DataFrame
Columns: [(, ID), (Age, Age), (Gender, Gender), (Diagnosis, Diagnosis), (Refractive_Defect, dioptre_1), (Refractive_Defect, dioptre_2), (Refractive_Defect, astigmatism), (Phakic/Pseudophakic, Phakic/Pseudophakic), (IOP, Pneumatic), (IOP, Perkins), (Pachymetry, Pachymetry), (Axial_Length, Axial_Length), (VF_MD, VF_MD)]
Index: []


In [201]:
#trying to disover any outliers - not working
import seaborn as sns
import matplotlib.pyplot as plt

# create a dataframe with the relevant columns
cols = ['ID', 'Age', 'Gender', 'Diagnosis', 'Refractive_Defect_dioptre_1', 'Refractive_Defect_dioptre_2', 'Refractive_Defect_astigmatism', 'Phakic/Pseudophakic', 'IOP_Pneumatic', 'IOP_Perkins', 'Pachymetry', 'Axial_Length', 'VF_MD']
df = od_clean.columns

# create boxplots for each numeric column

od_clean_age = od_clean[['Age']].tolist()
print(od_clean_age)
od_clean_age= pd.to_numeric(od_clean_age, errors='coerce')
numeric_cols = ['Age']
for col in numeric_cols:
    plt.figure()
    sns.boxplot(x=od_clean[col])
    plt.title(col)

# create scatterplots for each pair of numeric columns
sns.pairplot(df[od_clean.columns])

# create violin plots for each categorical column
categorical_cols = ['Gender', 'Diagnosis', 'Phakic/Pseudophakic', 'IOP_Pneumatic', 'IOP_Perkins']
for col in categorical_cols:
    plt.figure()
    sns.violinplot(x=col, y='Age', data=df)
    plt.title(col)



AttributeError: 'DataFrame' object has no attribute 'tolist'

In [ ]:
df_clean['Gender','Gender'] = df_clean['Gender','Gender'].replace({'male':0, 'female':1})


